In [1]:
from abc import ABC, abstractmethod

In [2]:
class Runnable(ABC):

  @abstractmethod
  def invoke(input_data):
    pass

In [5]:
import random
import warnings

class NakliLLM(Runnable):

    def __init__(self):
        print('LLM created')

    def invoke(self, prompt):
        response_list = [
            'Delhi is the capital of India',
            'IPL is a cricket league',
            'AI stands for Artificial Intelligence'
        ]
        return {'response': random.choice(response_list)}

    def predict(self, prompt):
        warnings.warn(
            "'predict' is deprecated and will be removed in future updates. "
            "Use 'invoke' instead.",
            DeprecationWarning,
            stacklevel=2
        )
        response_list = [
            'Delhi is the capital of India',
            'IPL is a cricket league',
            'AI stands for Artificial Intelligence'
        ]
        return {'response': random.choice(response_list)}


In [7]:
class NakliPromptTemplate(Runnable):

  def __init__(self, template, input_variables):
    self.template = template
    self.input_variables = input_variables

  def invoke(self, input_dict):
    return self.template.format(**input_dict)

  def format(self, input_dict):
    return self.template.format(**input_dict)

In [8]:
class NakliStrOutputParser(Runnable):

  def __init__(self):
    pass

  def invoke(self, input_data):
    return input_data['response']

In [9]:
class RunnableConnector(Runnable):

  def __init__(self, runnable_list):
    self.runnable_list = runnable_list

  def invoke(self, input_data):

    for runnable in self.runnable_list:
      input_data = runnable.invoke(input_data)

    return input_data

In [10]:
template = NakliPromptTemplate(
    template='Write a {length} poem about {topic}',
    input_variables=['length', 'topic']
)

In [11]:
llm = NakliLLM()

LLM created


In [12]:
parser = NakliStrOutputParser()

In [13]:
chain = RunnableConnector([template, llm, parser])

In [14]:
chain.invoke({'length':'long', 'topic':'india'})

'IPL is a cricket league'

In [15]:
template1 = NakliPromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

In [16]:
template2 = NakliPromptTemplate(
    template='Explain the following joke {response}',
    input_variables=['response']
)

In [17]:
llm = NakliLLM()

LLM created


In [18]:
parser = NakliStrOutputParser()

In [19]:
chain1 = RunnableConnector([template1, llm])

In [20]:
chain2 = RunnableConnector([template2, llm, parser])

In [21]:
final_chain = RunnableConnector([chain1, chain2])

In [22]:
final_chain.invoke({'topic':'cricket'})

'IPL is a cricket league'